In [3]:
# import ptvsd
# ptvsd.enable_attach(address = ('0.0.0.0', 35678))
# ptvsd.wait_for_attach()

import os,sys,time
from labeled_log_reader.tools import file_tools
from labeled_log_reader.parsing import drain_wrapper

from labeled_log_reader import config

import json
import pickle
from tqdm import tqdm
import csv
ENABLE_FAST_TEST = False
import multiprocessing
import re
import flinkdrain3



#! do not modify this function
def get_template_from_drain(drain_obj):
    """ 
    Dump template information from drain object
        Access to drain 
            _ = list(drain_obj.drain.id_to_cluster.values())[0]
            ### _ is drain3.drain.LogCluster object
            cluster_id = _.cluster_id
            template_string = _.get_template()
            size = _.size
    """
    
    
    
    return [
        {
            "cluster_id":_log_cluster.cluster_id,
            "template":_log_cluster.get_template(),
            "size":_log_cluster.size
        }
        for _log_cluster in drain_obj.drain.id_to_cluster.values()
    ]



SIMTH = 8
#* read thunderbird data
HADOOP_DATASET_FOLDER = config.HADOOP_DATASET_FOLDER


#* data reader



/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000002.log
/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000003.log
/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000012.log
/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000006.log
/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000007.log
/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000009.log
/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/application_1445062781478_0020/container_1445062781478_0020_01_000001.log
/home/lizongy

NotADirectoryError: [Errno 20] Not a directory: '/home/lizongyang/hdd-lizongyang/LOG_ARENA/original_datasets/Hadoop/abnormal_label.txt'

In [110]:
rregex  = re.compile(r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2},\d{3}) (INFO|WARN|ERROR) (\[.*?\]) ([\w.]*?): ((?:.|\n)*?)(?=^(?:\d{4}-\d{2}-\d{2}))",re.M)

allres = []
for dirname in tqdm(os.listdir(HADOOP_DATASET_FOLDER)):
    if dirname == "README.md" or dirname == "abnormal_label.txt":
        continue
    for filename in os.listdir(os.path.join(HADOOP_DATASET_FOLDER,dirname)):
        absfilepath = os.path.join(os.path.join(HADOOP_DATASET_FOLDER,dirname),filename)
        
        with open(absfilepath,'r') as fp:
            content = fp.read()
            res_list = rregex.findall(content+"0000-00-00")
        _res = [
            {
                'dirname':dirname,
                'filename':filename,
                'date':ttuple[0],
                'time':ttuple[1],
                'level':ttuple[2],
                'domain':ttuple[3],
                'str_pkg':ttuple[4],
                'str_content':ttuple[5]
            }
            for ttuple in res_list
        ]
        # file_tools.jsonline_save(_res,os.path.join(config.HADOOP_ASSET_DIR,"aggregate.jsonl"),append=True)
        allres.extend(_res)

100%|██████████| 57/57 [00:02<00:00, 27.44it/s]


In [111]:
infolog = [i['str_content'].split("\n")[0] for i in allres if i['level']=='INFO']
warnlog = [i['str_content'].split("\n")[0] for i in allres if i['level']=='WARN']
errorlog = [i['str_content'].split("\n")[0] for i in allres if i['level']=='ERROR']

In [114]:
drain_config_path = os.path.join(config.HADOOP_ASSET_DIR,"drain_Hadoop_simth08.ini")

def deal_loglist(loglist):
    config = flinkdrain3.template_miner_config.TemplateMinerConfig_ini_modified()
    config.load(drain_config_path)
    miner = flinkdrain3.TemplateMiner(None,config)
    for line in loglist:
        result = miner.add_log_message(line)
    rescontent = get_template_from_drain(miner)
    return [i['template'] for i in rescontent]

In [118]:
["- "+i for i in deal_loglist(warnlog)]

['- Could not delete',
 '- Unable to parse prior job history, aborting recovery',
 '- Could not parse the old history file. Will not have old AMinfos',
 '- DFSOutputStream ResponseProcessor exception for block BP---:blk_',
 '- Error Recovery for block BP---:blk_ in pipeline :, :: bad datanode :',
 '- Slow ReadProcessor read fields took (threshold=); ack: seqno: status: SUCCESS status: ERROR downstreamAckTimeNanos: , targets: [:, :]',
 '- I/O error constructing remote block reader.',
 '- Failed to connect to /: for block, add to deadNodes and continue. java.net.NoRouteToHostException: No route to host: no further information',
 '- DFS chooseDataNode: got # IOException, will wait for msec.',
 '- Address change detected. Old: msra-sa-/: New: msra-sa-:',
 '- DFS Read',
 '- Exception running child : java.net.NoRouteToHostException: No Route to Host from MININT-FNANLI5/ to msra-sa-: failed on socket timeout exception: java.net.NoRouteToHostException: No route to host: no further information;

In [ ]:
\njava.io.IOException

In [105]:
def mask(i):
    i = re.sub(r"(?:container)?[\d_.]{2,}","NUM",i)
    i = re.sub(r"(?<=\W)\w{1,}?(?:[A-z][0-9]|[0-9][A-z])\w{1,}?(?=\W)",'HASH',i)
    i = re.sub(r"hdfs:\/\/.*(?=\s|$)",'HDFS_PATH',i)
    # i = re.sub(r"(?<=(?:\W|^))[A-z]\d{2,}?(?=(?:\W|$))",'',i)
    # i = re.sub(r"(?<=(?:\W|^))\d{2,}?[A-z](?=(?:\W|$))",'',i)
    # i = re.sub(r"(?<=(?:\W|^))[A-z]\d{2,}?[A-z](?=(?:\W|$))",'',i)
    return i

In [97]:
list(set([mask(i['str_content'].split("\n")[0]) for i in allres if i['level']=='ERROR']))
# list(set([i['str_content'].split("\n")[0] for i in allres if i['level']=='ERROR']))

['Container complete event for unknown container id NUM',
 'Could not contact RM after NUM milliseconds.',
 'ERROR IN CONTACTING RM. ',
 'Error writing History Event: org.apache.hadoop.mapreduce.jobhistory.JobUnsuccessfulCompletionEvent@NUMbbNUMa',
 'Shuffle failed : local error on this node: HASH/NUM',
 'Error writing History Event: org.apache.hadoop.mapreduce.jobhistory.JobUnsuccessfulCompletionEvent@NUMdeNUMf',
 'Error closing writer for JobID: jobNUM',
 'Error writing History Event: org.apache.hadoop.mapreduce.jobhistory.TaskAttemptUnsuccessfulCompletionEvent@NUMd',
 'Exception while unregistering ',
 "Error communicating with RM: Resource Manager doesn't recognize AttemptId: applicationNUM",
 'Error writing History Event: org.apache.hadoop.mapreduce.jobhistory.JobUnsuccessfulCompletionEvent@NUMb',
 'Connection retry failed with 4 attempts in NUM seconds',
 'Thread Thread[eventHandlingThread,5,main] threw an Exception.',
 'Error communicating with RM: Could not contact RM after NUM

In [106]:
list(set([mask(i['str_content'].split("\n")[0]) for i in allres if i['level']=='WARN']))


['Failed to connect to /NUM:NUM for block, add to deadNodes and continue. java.net.ConnectException: Connection timed out: no further information',
 'Failed to renew lease for [DFSClient_NONMAPREDUCE_-NUM] for NUM seconds.  Will retry shortly NUM',
 'Failed to connect to /NUM:NUM for block, add to deadNodes and continue. java.net.NoRouteToHostException: No route to host: no further information',
 'When stopping the service JobHistoryEventHandler : org.apache.hadoop.yarn.exceptions.YarnRuntimeException: java.nio.channels.ClosedChannelException',
 'I/O error constructing remote block reader.',
 'Error Recovery for block BP-NUM-NUM-NUM:blkNUM in pipeline NUM:NUM, NUM:NUM: bad datanode NUM:NUM',
 'Exception running child : java.net.NoRouteToHostException: No Route to Host from  MININT-FNANLI5/NUM to msra-sa-NUM:NUM failed on socket timeout exception: java.net.NoRouteToHostException: No route to host: no further information; For more details see:  http://wiki.apache.org/hadoop/NoRouteToHost

In [107]:
a = list(set([mask(i['str_content'].split("\n")[0]) for i in allres if i['level']=='INFO']))
print(len(a))

2612


In [109]:
for i in a:
    print(i)

After Scheduling: PendingReds:0 ScheduledMaps:0 ScheduledReds:0 AssignedMaps:NUM AssignedReds:1 CompletedMaps:5 CompletedReds:0 ContAlloc:NUM ContRel:0 HostLocal:NUM RackLocal:2
After Scheduling: PendingReds:0 ScheduledMaps:0 ScheduledReds:0 AssignedMaps:8 AssignedReds:1 CompletedMaps:7 CompletedReds:0 ContAlloc:NUM ContRel:0 HostLocal:NUM RackLocal:3
 Using ResourceCalculatorProcessTree : org.apache.hadoop.yarn.util.WindowsBasedProcessTree@1eNUMec
Final Stats: PendingReds:0 ScheduledMaps:0 ScheduledReds:0 AssignedMaps:0 AssignedReds:1 CompletedMaps:NUM CompletedReds:0 ContAlloc:3 ContRel:0 HostLocal:2 RackLocal:0
After Scheduling: PendingReds:1 ScheduledMaps:4 ScheduledReds:0 AssignedMaps:6 AssignedReds:0 CompletedMaps:0 CompletedReds:0 ContAlloc:6 ContRel:0 HostLocal:2 RackLocal:4
Before Scheduling: PendingReds:0 ScheduledMaps:0 ScheduledReds:0 AssignedMaps:9 AssignedReds:1 CompletedMaps:5 CompletedReds:0 ContAlloc:NUM ContRel:1 HostLocal:NUM RackLocal:3
After Scheduling: PendingReds